> 本文由 [简悦 SimpRead](http://ksria.com/simpread/) 转码， 原文地址 http://mtoou.info/327.html

- [NVIDIA GPU视频编解码能力表](https://cloud.tencent.com/developer/article/1446037)

硬解还是软解，这是个问题。但关键是，你能硬解么？视频文件要播放得先拆包分离，然后对分离出的数据解码，解码后才是可播放的 raw 视频直接输出到显示器播放。其中使用 cpu 进行解码，就是软解，使用 GPU（[显卡](http://mtoou.info/tag/%e6%98%be%e5%8d%a1/ "浏览关于“显卡”的文章")）进行解码，就是硬解。

那么为什么需要硬解。一般来说，windows 下，默认的解码方式一般是软解，硬解是相对有点麻烦的。cpu 的运算能力足够，一般使用软解也非硬解，因为软解现在更成熟，对应的编码更广，也更稳定。但，前提是你的 cpu 足够劲。我认为拥有移动版 i3 以上的 cpu，硬解就毫无必要了。但我用来装 linux 的小贝壳是 c-60 的 cpu，频率只有坑爹的双核 1G, 还不如我的手机，但是它内置有一颗不错的显卡，HD6290，可以达到 i3 级别，这种情况下硬解就是唯一出路。但是 linux 硬解却是很麻烦的。

### Linxu 显卡[驱动](http://mtoou.info/tag/%e9%a9%b1%e5%8a%a8/ "浏览关于“驱动”的文章")

现在显卡制造商分为三大家：intel，ati 和 nvidia，一般来说，n 卡是最稳定支持也最多的，但是 n 卡只有闭源驱动，这也是林纳斯对 nvidia 竖起中指的原因之一，intel 次之，intel 只有一种驱动，但这玩意是开源的，配合 intel 强大的技术力，让 intel 成为至少是我认为的 linux 最好选择，最后是 ati，广大 linuxer 认为最坑爹的厂商。ati 有 2 种驱动，开源的一种，只能发挥出显卡一半的性能而且耗能极大，闭源的一种，相对好一点但比 n 卡稳定性差很多。我的就是 a 卡，至今为止不曾悲剧。

但是光有显卡驱动还不够. 这在 windows 上也是一样的。显卡驱动本身不提供硬解所用的编程接口，你还需要驱动和播放器中间的 api 接口，从 windows 上这个叫 DXVA, 是自带的，由微软定义。从 linux 上由于没有了厂家，所以只能由显卡制造商来定义，n 卡的名为 vadpu，a 卡和 intel 的名为 [vaapi](http://mtoou.info/tag/vaapi/ "浏览关于“vaapi”的文章")，是由 intel 开发的。

我的猜测, 可能不对 (我经常犯错). 在 windows 上编写一段代码，计算 1+1=?，很简单就得出了，但是这个结果是由 cpu 计算的, 如果要由 gpu 计算 1+1=?，就得安装相应厂家的 sdk，道理是一样的。相应显卡驱动可能提供了把某条线显示在某个位置的函数，但是不一定提供计算的函数，所以需要 sdk 把计算翻译成驱动能理解的语言。直接和驱动对话, 如果播放器自身具备翻译的能力应该也是可以的，但是不现实，要每个播放器都去实现这些复杂又相互重复的任务。所以厂家就站出来，提供了这段代码。所以：

> 1. 你需要正确安装显卡 (闭源) 驱动  
> 2. 你需要正确的安装 vaapi(或 vadpu)  
> 3. 你需要一个能正确调用 vaapi 接口的播放器

看起来是很艰巨的任务啊。

首先 cat /var/log/Xorg.0.log | grep UVD。如果 [2785.567] (II) fglrx(0): UVD feature is enabled(II) fglrx(0): 或者类似，说明硬件支持。我参照 http://taoofye.appspot.com/?p=204001(需那啥, 否则打不开) 和 http://taoofye.appspot.com/?p=204001 进行设置。但这两篇都太老了，跟不上时代的发展，前者虽然是最近的但是介绍的方法不奏效, 我来总结一下，唯有原理是永不过时的。首先你需要正确安装闭源驱动, 我的 ati 显卡，直接安装了 ubuntu 附加驱动里提供的版本，实测 OK。接下来是 vaapi, 我单纯的也是从源里安装：

> sudo apt-get install va-i965-driver  
> sudo apt-get install libva vainfo  
> sudo apt-get install xvba-va-driver  
> sudo apt-get install libva-glx1 libva-dev

会安装一些依赖包：

> libva：也就是传说中的 VAAPI，是一个开放源代码的提供硬件解码功能的模块；  
> va-i965-driver：Intel 显卡的 VAAPI 后端，负责连接 Intel 显示卡驱动与 VAAPI。  
> xvba-video：AMD/ATi 显卡的 VAAPI 后端，负责连接 AMD/ATi 显示卡驱动与 VAAPI。  
> vdpau：NV 显卡的 VAAPI 后端, 负责连接 NV 显示卡驱动与 VAAPI。

然后 vainfo 一下，如果最下面有 vaapi 且没有什么错误，就成功了。

### vaapi 接口播放器

linux 上可用硬解的播放器，据我调查，分为御三家。XBMC 可调用 xvba 硬解，大名鼎鼎的 mplayer，可调用 vaapi，和花见花开的 vlc，这好像是最好的。

XBMC 我不是太熟, 之前都没听说过, 有兴趣的可以看看 http://forum.xbmc.org/showthread.php?tid=116996, 有详细的说明, 需要注意的是它也要添加第三方源来安装, 不过有现成编译好的版本

Mplayer 本身的代码不含任何 vaapi 支持，气人的是它内置了 vaapu 支持，团队老大大约是 nvidia 的人。支持 vaapi 的是 mplayer 的一个分支，叫 mplayer-vaapi，你可以在 gitorious.org/vaapi/mplayer 使用 git 获得源代码，可是这货是需要自己编译的，我花了整整一个晚上编译它，最后碰了一鼻子灰。和网上说的完全不一样。make 失败，我怎么知道 vo_vaapi.c 里为什么他妹的少个变量，这又不像少个文件那么简单。需要注意的是这货在 pacman 里有一个预编译的包，所以假如你使用 arch 的话，可以试试。而且这货即使安装成功也没有出色的图形前端，公认最好的前端 smplayer 在 vaapi 解码下各种死机。但如果能忍受这一切并且编译成功的话，据我所知它是性能最好的。

最后是 vlc，最傻瓜的选择。最新的 vlc 已经预先内置了 vaapi 支持，所以你直接在 vlc 里输入与解码器选项卡，打开硬件加速，就可以硬解了。如果是从命令行启动 vlc 可以看到 using va-api 0.32 version 等等类似的信息就行了。需要注意的是 vlc 的 wiki 里有两篇谈到硬解如何如何费事，还要重新编译 vlc 云云那都是三年前的，现在不那样了。

那么硬解有什么效果？我的华硕 1015BX 之前播放 720p 偶尔会卡，现在完全不卡了，下载了《我是传奇》的 1080P 官方宣传片，也不卡，但是不是所有 1080P 都不卡，放了放思路压得拯救大兵瑞恩，每隔 5 秒会卡一下，但起码不是完全不能放，证明解码能力的确大大提高，与此同时，cpu 占用大概只下降了 10% 还不到，但是解码能力提高是关键。

我很希望在买这小白贝壳机之前有人能告诉我这些，但是事实证明，世上没有什么救世主，一切还是得靠自己。